In [7]:
import sys, pathlib
import pandas as pd
import numpy as np
import joblib

# Añadir src/ al path para poder importar config
PROJECT_ROOT = pathlib.Path().resolve().parent.parent  
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src import config as cfg


In [9]:
# Leer precios desde parquet
df = pd.read_parquet(cfg.DATA / "raw" / "prices.parquet")
df = df.ffill().dropna()

# Calcular retornos logarítmicos
ret = np.log(df / df.shift(1)).dropna()
ret.head()

Ticker,AAPL,ABT,ADBE,AMZN,BAC,BTC-USD,COST,CRM,CSCO,CVX,...,PFE,PG,T,TSLA,UNH,V,VZ,WFC,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2012-05-21,0.056626,0.003890,0.022111,0.019725,-0.027439,0.0,0.009480,0.023556,0.012071,0.012415,...,-0.001330,-0.002049,-0.000892,0.042968,0.028485,0.031288,-0.004585,0.014758,0.009724,0.006972
2012-05-22,-0.007708,0.002908,0.000000,-0.012828,0.021725,0.0,-0.004189,0.002680,0.003593,-0.003719,...,-0.007571,-0.003793,-0.002680,0.068181,0.003055,0.018500,0.001209,0.008562,0.010886,-0.001097
2012-05-23,0.024107,-0.010052,0.005297,0.009015,0.026856,0.0,0.002464,0.007732,-0.002394,-0.003227,...,-0.012596,-0.012108,-0.005381,0.007118,-0.006482,0.006567,-0.002661,0.002208,0.013249,0.001097
2012-05-24,-0.009226,0.016965,-0.020089,-0.009433,-0.004193,0.0,0.013947,-0.029650,-0.018138,0.010650,...,0.002261,0.002881,0.008358,-0.024145,0.015415,0.005022,0.002661,0.002203,0.007559,0.006924
2012-05-25,-0.005374,-0.002728,0.001901,-0.010978,0.001400,0.0,0.000000,0.005389,-0.003667,-0.012065,...,-0.000452,-0.001279,0.001485,-0.015644,-0.001780,-0.003346,0.001449,0.001570,0.003682,-0.006436


In [13]:
# Retorno **medio** 5 días
ret5 = ret.rolling(5).mean()

# Volatilidad 5 días
vol5 = ret.rolling(5).std()

# Momentum = retorno / volatilidad
momentum = (ret5 / (vol5 + 1e-6)).clip(-10, 10)

# Target: retorno medio 5d futuro (t+1 … t+5)
y = ret5.shift(-5).dropna()

# Alinear features al target
ret = ret.shift(1).reindex(y.index)
momentum = momentum.shift(1).reindex(y.index)

# Visual check
ret.tail(3), momentum.tail(3), y[-3:]


(Ticker          AAPL       ABT      ADBE      AMZN       BAC   BTC-USD  \
 Date                                                                     
 2025-06-19  0.004793  0.000982 -0.012199 -0.010764  0.018592  0.002694   
 2025-06-20  0.000000  0.000000  0.000000  0.000000  0.000000 -0.001900   
 2025-06-21  0.022235  0.004371 -0.002967 -0.013406  0.009717 -0.013219   
 
 Ticker          COST       CRM      CSCO       CVX  ...       PFE        PG  \
 Date                                                ...                       
 2025-06-19 -0.002888 -0.010900  0.007776 -0.004444  ... -0.005013 -0.003159   
 2025-06-20  0.000000  0.000000  0.000000  0.000000  ...  0.000000  0.000000   
 2025-06-21  0.005513  0.004345  0.007264  0.009136  ...  0.003762  0.006686   
 
 Ticker             T      TSLA       UNH         V       VZ       WFC  \
 Date                                                                    
 2025-06-19  0.000362  0.017858 -0.005421 -0.050023 -0.00431  0.030429   

In [15]:
X = []
for i in range(cfg.WINDOW, len(y)):
    ventana_ret = ret.iloc[i - cfg.WINDOW:i].values       # (60, N)
    ventana_mom = momentum.iloc[i - cfg.WINDOW:i].values  # (60, N)
    bloque = np.concatenate([ventana_ret, ventana_mom], axis=1)  # (60, 2N)
    X.append(bloque)

X = np.stack(X)  # shape = (n_muestras, 60, 2N)
y = y.iloc[cfg.WINDOW:].values  # shape = (n_muestras, N)

print(f"✅ X shape: {X.shape}")
print(f"✅ y shape: {y.shape}")


✅ X shape: (4455, 60, 80)
✅ y shape: (4455, 40)


In [25]:
# Filtrar ventanas con algún NaN
X_clean, y_clean = [], []
for i in range(len(X)):
    if not np.isnan(X[i]).any():
        X_clean.append(X[i])
        y_clean.append(y[i])

X = np.stack(X_clean)
y = np.stack(y_clean)

print(f"✅ X limpio: {X.shape}")
print(f"✅ y limpio: {y.shape}")

✅ X limpio: (4450, 60, 80)
✅ y limpio: (4450, 40)


In [27]:
# Ver una muestra
pd.DataFrame(X[0]).iloc[:5]  # primeros 5 días de la primera muestra

# Ver un target
pd.Series(y[0], index=df.columns).sort_values(ascending=False).head()


Ticker
NKE      0.006007
COST     0.003281
AMZN     0.003280
GOOGL    0.003173
WMT      0.002767
dtype: float64

In [29]:
joblib.dump({
    "X": X,
    "y": y,
    "tickers": df.columns.tolist()
}, cfg.DATA / "processed" / "gru5d_data.pkl")


loaded = joblib.load(cfg.DATA / "processed" / "gru5d_data.pkl")
print("Tickers guardados:", len(loaded["tickers"]))

Tickers guardados: 40
